In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import os,sys
import utilities
import matplotlib.pyplot as plt
from numpy import array
from multiprocessing import Pool
import multiprocessing as mp
from consts import sensor_lat as lat
from consts import sensor_log as log
from consts import valid_sensor_res_10 as valid_sensor_list
from consts import valid_sensor_res_10_old as valid_sensor_list_old
from consts import sensor_day_seconds as day_seconds
from consts import sensor_week_seconds as week_seconds 
from consts import sensor_start_time_unix as start_time
from consts import sensor_end_time_unix as end_time
from consts import sensor_epoch_length as epoch_length
from consts import sensor_optimal_time_interval as optimal_time_interval
from consts import sensor_loc_pic as loc_pic
from consts import sensor_drop_list as drop_list
import consts
import time
import pickle
import copy
import h5py


In [22]:
import torch.utils.data as Data
import torch
from torch import nn,optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [2]:
from ss_utilities import dataset_read
from ss_utilities import dataset_write


In [16]:
time_stamp = np.arange(start_time,end_time+optimal_time_interval,optimal_time_interval)
unix_column = 'Time since the epoch [s]'
idd = 'Station ID'
a_temperature = 'Ambient Temperature'
s_temperature = 'Surface Temperature'
data_index = ['a_temperature_mean','a_temperature_std','a_temperature_array','sub_df_list_optimal_index','time_stamp',]
df_dict = {'rb_optimal':5959,'optimal':5959,'df_list':20160}

In [17]:
with h5py.File('dataset.hdf5','r') as f:
     a_temperature_mean = f[data_index[0]][:]
     a_temperature_std = f[data_index[1]][:]
     a_temperature_array = f[data_index[2]][:]
     sub_df_list_optimal_index = f[data_index[3]][:]
     time_stamp = f[data_index[4]][:]

In [31]:
class DiscriminatorNet(torch.nn.Module):
    def __init__(self):
        super(DiscriminatorNet,self).__init__()
        n_features = 44
        n_out = 1
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features,512),
            nn.LeakyReLU(0.2),
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(512,256),
            nn.LeakyReLU(0.2),
        )
        self.out = nn.Sequential(
            nn.Linear(256,n_out),
            nn.Sigmoid()
        )
    def forward(self,x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.out(x)
        return x

In [79]:
dis = DiscriminatorNet()

In [74]:
class GeneratorNet(torch.nn.Module):
    def __init__(self):
        super(GeneratorNet,self).__init__()
        n_features = 100
        n_out = 44

        self.hidden0 = nn.Sequential(
            nn.Linear(n_features,256),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(256,512),
            nn.LeakyReLU(0.2),
        )
        self.out = nn.Sequential(
            nn.Linear(512,n_out),
            nn.Tanh()
        )
    def forward(self,x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.out(x)
        return x

In [80]:
gen = GeneratorNet()

In [68]:
def noise(size):
    n = Variable(torch.randn(size,100,dtype=torch.float32))
    return n

In [69]:
noise(44).shape

torch.Size([44, 100])

In [71]:
n1 = noise(44)

In [76]:
n12 = generator(n1)

In [81]:
loss = nn.BCELoss()

In [83]:
def ones_target(size):
    data = Variable(torch.ones(size,1,dtype=torch.float32))
    return data

def zeros_target(size):
    data = Variable(torch.zeros(size,1,dtype=torch.float32))
    return data

In [84]:
def train_discriminator(optimizer,real_data,fake_data):
    optimizer.zero_grad()

    N = real_data.size(0)

    prediction_real = dis(real_data)
    error_real = loss(prediction_real,ones_target(N))
    error_real.backward()

    prediction_fake = dis(fake_data)
    error_fake = loss(prediction_fake,zeros_target(N))

    error_fake.backward()
    
    optimizer.step()

    return error_real+error_fake, prediction_real, prediction_fake

In [86]:
def train_generator(optimizer,fake_data):
    N = fake_data.size(0)

    optimizer.zero_grad()

    prediction = dis(fake_data)

    error = loss(prediction,ones_target(N))

    error.backward()

    optimizer.step()
    return error

In [87]:
num_epochs = 200

for epoch in range(num_epochs):
    for n_batch,(real_batch) in enumerate(train_dataloader):
        N = real_batch.size(0)

        real_batch = real_batch - real_batch.mean(axis=0)
        real_data = Variable(real_batch)

        

SyntaxError: unexpected EOF while parsing (<ipython-input-87-d8146acb3ad1>, line 3)